# Lab4 - Predicting Binary Outcomes
## Logistic Regression, LDA, and QDA on civil war outbreaks

In [2]:
import pandas as pd
import statsmodels.api as sm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA


### Question 1

#### Fit a logistic regression model for the start of civil war on all other variables except country and year (yes, this makes some questionable assumptions about independent observations); include a quadratic term for exports. Report the coefficients and their standard errors, together with R's p-values. Which ones does R (Python) say are significant at the 5% level?


In [7]:
war = pd.read_csv("http://www.stat.cmu.edu/~cshalizi/uADA/15/hw/06/ch.csv")
war = war.ix[:,1:len(war.columns)]
war = war.dropna()

war.head()


,country,year,start,exports,schooling,growth,peace,concentration,lnpop,fractionalization,dominance
9,Algeria,1965,0.0,0.190,10.0,-1.682,24.0,0.916,16.293980,132.0,1.0
10,Algeria,1970,0.0,0.193,16.0,2.843,84.0,0.916,16.436260,132.0,1.0
11,Algeria,1975,0.0,0.269,26.0,4.986,144.0,0.916,16.589220,132.0,1.0
12,Algeria,1980,0.0,0.368,40.0,3.261,204.0,0.916,16.742380,132.0,1.0
13,Algeria,1985,0.0,0.170,59.0,1.602,264.0,0.916,16.901039,132.0,1.0


In [8]:
warX = war.drop(["country","year","start"], axis=1)
warX["exports^2"] = warX["exports"]**2
warX["intercept"] = 1.0
warY = war["start"]



m1 = sm.Logit(warY, warX).fit()
print(m1.summary())

Optimization terminated successfully.
         Current function value: 0.186355
         Iterations 9
                           Logit Regression Results                           
Dep. Variable:                  start   No. Observations:                  688
Model:                          Logit   Df Residuals:                      678
Method:                           MLE   Df Model:                            9
Date:                Sat, 24 Jun 2017   Pseudo R-squ.:                  0.2407
Time:                        19:15:19   Log-Likelihood:                -128.21
converged:                       True   LL-Null:                       -168.86
                                        LLR p-value:                 8.902e-14
                        coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
exports              18.9370      5.865      3.229      0.001         7.441    30.433
schoolin

In [6]:
type(warY)

pandas.core.series.Series

See table above for coefficients, SEs, and p-values.
All but dominance are significant at the 5% level, although dominance is close: p = 0.058.

### Question 2
#### Interpretation: All parts of this question refer to the logistic regression model you just fit.

#### What is the model's predicted probability for a civil war in India in the period beginning 1975?

In [3]:
warBase = war.copy()
warBase["exports^2"] = war["exports"]**2
warBase["intercept"] = 1.0

war_India_1975 = warBase[(war.country == "India") & (war.year == 1975)]
war_India_1975 = war_India_1975.drop(["country","year","start"], axis=1)

m1.predict(war_India_1975)[0]

0.3504198862846219

#### What probability would it predict for a country just like India in 1975, except that its male secondary school enrollment rate was 30 points higher?

In [4]:
war_India_1975_school = war_India_1975.copy()
war_India_1975_school["schooling"] = war_India_1975_school["schooling"] + 30

m1.predict(war_India_1975_school)[0]

0.17309004735035405

#### What probability would it predict for a country just like India in 1975, except that the ratio of commodity exports to GDP 
 was 0.1 higher?

In [5]:
war_India_1975_exports = war_India_1975.copy()
war_India_1975_exports["exports"] = war_India_1975_exports["exports"] + 0.1
war_India_1975_exports["exports^2"] = war_India_1975_exports["exports"]**2

m1.predict(war_India_1975_exports)[0]

0.69613783776825489

#### What is the model's predicted probability for a civil war in Nigeria in the period beginning 1965?

In [6]:
war_Nigeria_1965 = warBase[(war.country == "Nigeria") & (war.year == 1965)]
war_Nigeria_1965 = war_Nigeria_1965.drop(["country","year","start"], axis=1)

m1.predict(war_Nigeria_1965)[0]

0.17099171522633955

#### What probability would it predict for a country just like Nigeria in 1965, except that its male secondary school enrollment rate was 30 points higher?

In [10]:
war_Nigeria_1965_school = war_Nigeria_1965.copy()
war_Nigeria_1965_school["schooling"] = war_Nigeria_1965_school["schooling"] + 30

m1.predict(war_Nigeria_1965_school)[0]

0.074103148309710418

#### What probability would it predict for a country just like Nigeria in 1965, except that the ratio of commodity exports to GDP was 0.1 higher?

In [11]:
war_Nigeria_1965_exports = war_Nigeria_1965.copy()
war_Nigeria_1965_exports["exports"] = war_Nigeria_1965_exports["exports"] + 0.1
war_Nigeria_1965_exports["exports^2"] = war_Nigeria_1965_exports["exports"]**2

m1.predict(war_Nigeria_1965_exports)[0]

0.33100439923140973

#### Q:In the parts above, you changed the same predictor variables by the same amounts. If you did your calculations properly, the changes in predicted probabilities are not equal. Explain why not. (The reasons may or may not be the same for the two variables.)


A:
The different changes are due to using logistic regression rather than linear regression: because the function does not
 have a constant slope, the response to change depends on where in the data the observations are. 



### Question 3
#### Build a 2x2 confusion matrix (a.k.a. A classification tableor a contigency table) which counts: the number of outbreaks of civil war correctly predicted by the logistic regression; the number of civil wars not predicted by the model; the number of false predictions of civil wars; and the number of correctly predicted absences of civil wars. (Note that some entries in the table may be zero.)

In [12]:
my_log_pred = pd.DataFrame()
my_log_pred["pred"] = ["No" if x < .5 else "Yes" for x in m1.predict()]
my_log_pred["actual"] = ["No" if x == 0 else "Yes" for x in war["start"]]
conf_log = pd.crosstab(my_log_pred["pred"], my_log_pred["actual"])
conf_log

actual,No,Yes
pred,,
No,637,43
Yes,5,3


#### What fraction of the logistic regression's predictions are incorrect, i.e. what is the misclassification rate? (Note that this is if anything too kind to the model, since it's looking at predictions to the same training data set)

In [14]:
(1/(war.shape[0])) * (conf_log.iloc[1,0] + conf_log.iloc[0,1])

0.069767441860465115

#### Consider a foolish (?) pundit who always predicts no war. What fraction of the pundit's predictions are correct on the whole data set?

In [15]:
conf_log.iloc[:,0].sum()/war.shape[0]

0.9331395348837209

### Question 4
#### Comparison: Since this is a classification problem with only two classes, we can compare Logistic Regression right along side Discriminant Analysis.

#### Fit an LDA model using the same predictors that you used for your logistic regression model. What is the training misclassification rate?

In [19]:
lda1 = LDA(solver="svd", store_covariance=True)
lda1.fit(warX,warY)

my_lda_pred = pd.DataFrame()
my_lda_pred["pred"] = ["No" if x == 0 else "Yes" for x in lda1.predict(warX)]
my_lda_pred["actual"] = ["No" if x == 0 else "Yes" for x in war["start"]]
conf_lda = pd.crosstab(my_lda_pred["pred"], my_lda_pred["actual"])
conf_lda

C:\Users\dyoung\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:389: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


actual,No,Yes
pred,,
No,636,40
Yes,6,6


In [18]:
(1/(war.shape[0])) * (conf_lda.iloc[1,0] + conf_lda.iloc[0,1])

0.066860465116279064

#### Fit a QDA model using the very same predictors. What is the training misclassification rate?

<font color="blue">**R vs Python Note: ** I have not gotten QDA to work for this dataset in Python.</font>

In [22]:
qda1 = QDA(store_covariances=True)
qda1.fit(warX,warY)

test = qda1.predict_proba(warX)

my_qda_pred = pd.DataFrame()
my_qda_pred["pred"] = ["No" if x < .5 else "Yes" for x in qda1.predict(warX)]
my_qda_pred["actual"] = ["No" if x == 0 else "Yes" for x in war["start"]]
# conf_qda = pd.crosstab(my_qda_pred["pred"], my_qda_pred["actual"])
# conf_qda

# (1/(war.shape[0])) * (conf_qda.iloc[1,0] + conf_qda.iloc[0,1])

my_qda_pred["pred"]

C:\Users\dyoung\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:694: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0      No
1      No
2      No
3      No
4      No
5      No
6      No
7      No
8      No
9      No
10     No
11     No
12     No
13     No
14     No
15     No
16     No
17     No
18     No
19     No
20     No
21     No
22     No
23     No
24     No
25     No
26     No
27     No
28     No
29     No
       ..
658    No
659    No
660    No
661    No
662    No
663    No
664    No
665    No
666    No
667    No
668    No
669    No
670    No
671    No
672    No
673    No
674    No
675    No
676    No
677    No
678    No
679    No
680    No
681    No
682    No
683    No
684    No
685    No
686    No
687    No
Name: pred, dtype: object